In [ ]:
# Let's check that Colab is still running CUDA 10.1:
!nvcc --version

# OpenMM will attempt to use a GPU if it recognizes CUDA, which it always will
# on Colab regardless of whether a GPU is available, so we must select CPU vs
# GPU another way. Here we'll use the output from the nvidia-smi command:
from subprocess import Popen, PIPE
p = Popen(["nvidia-smi"], stdout = PIPE)
out = p.communicate()[0]
if "failed" in out.decode():
    use_gpu = False
else:
    use_gpu = True
    # Display nvidia-smi output so we can see what GPU we are using:
    print("\n\n" + out.decode())

# OpenMM will in general run much faster on a Colab GPU than a single processor.
# To use a GPU, go to Runtime > Change runtime type and request a GPU as the
# hardware accelerator.

print("\n\nUsing GPU? " + str(use_gpu))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Using GPU? False


In [ ]:
import sys
from urllib.request import urlretrieve

# Check if Colab is still running Python 3.6.x:
if not (sys.version_info.major == 3 and sys.version_info.minor == 6):
    print("This notebook is specific to Python 3.6.x")
    sys.exit(1)

# Add conda package install location to PATH so we can import OpenMM in Python:
sys.path.append("/usr/local/lib/python3.6/site-packages")

# Download Miniconda installer, specifically the last version targeting 3.6.x:
urlretrieve("https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64."
            "sh", filename = "miniconda.sh")
# Install Miniconda:
!bash miniconda.sh -bfp /usr/local/
# Use conda to install OpenMM, OpenForceField, and the analysis tool MDTraj:
!conda config --add channels conda-forge --add channels omnia --add channels omnia/label/cuda101
!conda install openmm mdtraj python=3.6 -y
!conda install openmmforcefields python=3.6 -y
# Install 3Dmol.js for viewing structures (unfortunately, we cannot use the more
# convenient nglviewer in Colab as of August 2020):
!pip install py3Dmol

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c

In [ ]:
# Download ideal imatinib structure from the RCSB PDB:
urlretrieve("https://files.rcsb.org/ligands/view/STI_ideal.sdf",
            filename = "imatinib.sdf")
urlretrieve("https://files.rcsb.org/ligands/view/STI_ideal.pdb",
            filename = "imatinib.pdb")

# For some reason, this conda installation of openforcefields doesn't seem to
# include the force field xml files themselves...
urlretrieve("https://raw.githubusercontent.com/openforcefield/openforcefields"
            "/master/openforcefields/offxml/openff-1.2.0.offxml",
            filename =  "openff-1.2.0.offxml")
urlretrieve("https://raw.githubusercontent.com/openmm/openmmforcefields"
            "/master/amber/ffxml/tip3p_standard.xml",
            filename =  "tip3p_standard.xml")

from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from openforcefield.topology import Molecule
molecule = Molecule.from_file("imatinib.sdf")

# Let the parameters for imatinib be determined from the OpenForceField 1.2 set:
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
smirnoff = SMIRNOFFTemplateGenerator(molecules = molecule,
                                     forcefield = "openff-1.2.0.offxml")
forcefield = ForceField("tip3p_standard.xml")
forcefield.registerTemplateGenerator(smirnoff.generator)
pdb = PDBFile("imatinib.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
# Create a periodic box, leaving at least 12 angstroms space around imatinib,
# and fill it with TIP3P waters, plus ions to bring the ionic strength to
# roughly physiological:
modeller.addSolvent(forcefield, model = "tip3p", padding = 12*angstroms,
                    ionicStrength=0.15*molar)
# Specify that we want to constrain hydrogen bond lengths (letting us use a
# longer time step), and use Particle Mesh Ewald:
system = forcefield.createSystem(modeller.topology, nonbondedMethod = PME,
                                 nonbondedCutoff = 10*angstroms,
                                 constraints = HBonds)

In [ ]:
from sys import stdout

if use_gpu:
    platform = Platform.getPlatformByName("CUDA")
    npt_steps = 500000 # 1 nanosecond
else:
    platform = Platform.getPlatformByName("CPU")
    npt_steps = 50000 # 100 picoseconds

# We'll use the LangevinIntegrator to propagate the system at a constant
# temperature, and a 2 femtosecond time step:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator, platform)
simulation.context.setPositions(modeller.positions)

simulation.minimizeEnergy() # minimize the total energy of the initial structure
simulation.step(5000) # equilibrate in NVT for 10 picoseconds
# add a barostat for NPT dynamics:
simulation.system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))

# write out coordinates and basic info 50 times:
simulation.reporters.append(app.PDBReporter("trajectory.pdb", npt_steps // 50))
simulation.reporters.append(app.StateDataReporter(stdout, npt_steps // 50,
                                                  step = True,
                                                  potentialEnergy = True,
                                                  temperature = True))

simulation.step(npt_steps) # simulate in NPT

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
6000,-108262.86529709464,294.2451388967849
7000,-108033.44510310321,298.3913622512366
8000,-108089.03158854808,298.76256549141783
9000,-108381.03198015291,303.73731837662393
10000,-108329.49872566562,300.3438226140909
11000,-107843.59457723657,298.0506058941754
12000,-108540.67916816912,302.1930289031705
13000,-108836.98972919848,294.17686282624345
14000,-108480.47492067679,300.50946166746644
15000,-108568.7569791131,298.19257594482207
16000,-107697.09375270247,298.8813593827424
17000,-108330.43905958036,302.65416362020596
18000,-108115.86763812909,302.81604329109683
19000,-108230.85645166636,304.5331535819598
20000,-108421.0062363774,306.8465528458886
21000,-107872.10321357338,298.50671378983805
22000,-108700.7823304342,302.4094223666153
23000,-108340.52291576294,299.4848853586935
24000,-109002.19800500317,299.8280045359068
25000,-108829.20143250439,300.4505476714405
26000,-108158.9218753753,302.75885988182506
27000,-107998.099789

In [ ]:
import py3Dmol

# Let's manually split the pdb by frame so we can show just the ending frame:
frames = []
with open("trajectory.pdb", 'r') as f:
    for line in f.readlines()[2:]: # remove REMARK and CRYST1
        if "MODEL" in line:
            frames.append("")
        else:
            frames[-1] += (line)

view = py3Dmol.view()
view.addModel(frames[-1], "pdb") # show the last frame (hydrogens are removed)
view.setStyle({'sphere':{}}) # show atoms as spheres
view.zoomTo() # center the view
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
import mdtraj as md

trajectory = md.load("trajectory.pdb")
# Get the atoms corresponding to imatinib:
imatinib_atoms = [atom for atom in trajectory.topology.atoms
                  if atom.residue.name == "STI"]
# Re-image the box so imatinib is centered:
trajectory_centered = trajectory.image_molecules(anchor_molecules = 
                                                 [imatinib_atoms])
# Write out just imatinib for now:
trajectory_centered.atom_slice([atom.index
                                for atom in
                                imatinib_atoms]).save("trajectory_centered.pdb")

In [ ]:
view = py3Dmol.view()
with open("trajectory_centered.pdb", 'r') as f:
    view.addModelsAsFrames(f.read(), "pdb")
view.setStyle({'stick':{}})
view.animate({"loop": "forward", "interval": 750}) # show a frame every 0.75 s
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Troubleshooting lack of OpenForceField XML files:

#from openforcefield.typing.engines.smirnoff.forcefield import get_available_force_fields
#get_available_force_fields()
#print(SMIRNOFFTemplateGenerator.INSTALLED_FORCEFIELDS)
#!find / -name openff-1.2.0.offxml